## Данные

Данные в [архиве](https://drive.google.com/file/d/15o7fdxTgndoy6K-e7g8g1M2-bOOwqZPl/view?usp=sharing). В нём два файла:
- `news_train.txt` тренировочное множество
- `news_test.txt` тренировочное множество

С некоторых новостных сайтов были загружены тексты новостей за период  несколько лет, причем каждая новость принаделжит к какой-то рубрике: `science`, `style`, `culture`, `life`, `economics`, `business`, `travel`, `forces`, `media`, `sport`.

В каждой строке файла содержится метка рубрики, заголовок новостной статьи и сам текст статьи, например:

>    **sport**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею разгромила чехов**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею крупно об...**

# Задача

1. Обработать данные, получив для каждого текста набор токенов
Обработать токены с помощью (один вариант из трех):
    - pymorphy2
    - русского [snowball стеммера](https://www.nltk.org/howto/stem.html)
    - [SentencePiece](https://github.com/google/sentencepiece) или [Huggingface Tokenizers](https://github.com/huggingface/tokenizers)
    
    
2. Обучить word embeddings (fastText, word2vec, gloVe) на тренировочных данных. Можно использовать [gensim](https://radimrehurek.com/gensim/models/word2vec.html) . Продемонстрировать семантические ассоциации. 

3. Реализовать алгоритм классификации, посчитать точноть на тестовых данных, подобрать гиперпараметры. Метод векторизации выбрать произвольно - можно использовать $tf-idf$ с понижением размерности (см. scikit-learn), можно использовать обученные на предыдущем шаге векторные представления, можно использовать [предобученные модели](https://rusvectores.org/ru/models/). Имейте ввиду, что простое "усреднение" токенов в тексте скорее всего не даст положительных результатов. Нужно реализовать два алгоритмов из трех:
     - SVM
     - наивный байесовский классификатор
     - логистическая регрессия
    

4.* Реализуйте классификацию с помощью нейросетевых моделей. Например [RuBERT](http://docs.deeppavlov.ai/en/master/features/models/bert.html) или [ELMo](https://rusvectores.org/ru/models/).

##  Импорты и препроцессинг

In [85]:
import random
import re
from tqdm.notebook import tqdm
from collections import Counter
from nltk.stem.snowball import RussianStemmer 
from nltk.corpus import stopwords


In [86]:
train_lines = list(open('../data/news_train.txt', 'r', encoding='utf-8'))
test_lines = list(open('../data/news_test.txt', 'r', encoding='utf-8'))
stop_words = set(stopwords.words("russian"))
random.shuffle(train_lines)
random.shuffle(test_lines)
train_lines[0]

'sport\tРядом с олимпийским стадионом в Рио начались протесты\tВ Рио-де-Жанейро рядом со стадионом «Маракана» накануне начала Олимпийских Игр проходит акция протеста, сообщает корреспондент «Ленты.ру». Полиция применила против демонстрантов слезоточивый газ.Как передает ТАСС, манифестанты, выступающие против использования бюджетных средств на организацию Олимпиады в Бразилии, подожгли национальный флаг и с его помощью имитировали эстафету олимпийского огня.Торжественная церемония открытия Олимпиады стартовала в 20:00 (02:00 мск). Игры в Рио будут проходить до 21 августа.\n'

In [87]:
'https://drive.google.com/file/d/1mG3tPS_59pANrgwd6T2IgnHWgph4vYbg/view?usp=sharing'

'https://drive.google.com/file/d/1mG3tPS_59pANrgwd6T2IgnHWgph4vYbg/view?usp=sharing'

In [88]:
def preprocess(lines):
    data = []
    for line in lines: 
        dct = {}
        lst = line.split('\t')
        dct['class'] = lst[0]
        dct['header'] = re.findall(r'\b\w+\b', lst[1].lower())
        dct['header'] = [w for w in dct['header'] if not w in stop_words]
        sentance_list = re.split(r"[.!?]", lst[2].lower())
        text = []
        for sen in sentance_list: 
            words = re.findall(r'\b\w+\b', sen.lower())
            words = [w for w in words if not w in stop_words]
            if len(words) > 1: 
                text.append(words)
        dct['text'] = text
        data.append(dct)
    return data

In [89]:
train_data = preprocess(train_lines)
test_data = preprocess(test_lines)

In [90]:
def stem_data(data):
    rs = RussianStemmer()
    for d in tqdm(data): 
        d['header'] = [rs.stem(word) for word in d['header']]
        d['text'] = [[rs.stem(word) for word in sentence] for sentence in d['text']]    

In [91]:
stem_data(train_data)

In [92]:
stem_data(test_data)

In [93]:
print(train_data[4])

{'class': 'economics', 'header': ['лукойл', 'реш', 'прода', 'сво', 'пенсион', 'фонд'], 'text': [['крупн', 'росс', 'частн', 'нефтян', 'компан', 'лукойл', 'ищет', 'покупател', 'принадлежа', 'пенсион', 'фонд', 'лукойл', 'гарант'], ['номер', '18', 'июл', 'пишет', 'газет', 'ведом', 'ссылк', 'неназва', 'участник', 'рынк'], ['сведен', 'изначальн', 'нпф', 'лукойл', 'гарант', 'прос', 'миллиард', 'доллар', 'однак', 'встреч', 'потенциальн', 'покупател', 'цен', 'сниж', 'втро'], ['дан', 'издан', 'соответств', 'переговор', 'ведет', 'управля', 'лукойл', 'гарант', 'ифд', 'капитал', 'принадлежа', 'виц', 'президент', 'лукойл', 'леонид', 'федун'], ['однак', 'председател', 'совет', 'директор', 'нпф', 'лукойл', 'гарант', 'александр', 'жирк', 'заяв', 'издан', 'известн', 'так', 'переговор'], ['указыва', 'собеседник', 'газет', 'лукойл', 'гарант', 'предлага', 'куп', 'фонд', 'миха', 'прохоров', 'онэкс', 'однак', 'последн', 'отказа', 'неясн', 'перспект', 'пенсион', 'реформ', 'котор', 'последн', 'врем', 'обсужда'

In [94]:
def header_to_text(data):
    for obj in data: 
        obj['text'].insert(0, obj['header'])
        del obj['header']
    return data

In [95]:
train_concat = header_to_text(train_data)
test_concat = header_to_text(test_data)

In [96]:
test_concat[:2]

[{'class': 'sport',
  'text': [['бывш', 'сотрудник', 'роснефт', 'возглав', 'российск', 'биатлон'],
   ['бывш',
    'руководител',
    'спортивн',
    'клуб',
    'роснефт',
    'александр',
    'кравц',
    'избра',
    'пост',
    'президент',
    'союз',
    'биатлонист',
    'росс',
    'срок',
    '2018',
    'год'],
   ['сообща', 'р', 'спорт'],
   ['должност',
    'директор',
    'центр',
    'спортивн',
    'подготовк',
    'сборн',
    'команд',
    'росс',
    'смен',
    'миллиардер',
    'миха',
    'прохоров',
    'котор',
    'пода',
    'отставк',
    'апрел',
    'год'],
   ['выбор', 'прошл', '20', 'ма', 'отчетн', 'конференц', 'сбр', 'москв'],
   ['польз',
    'кравцов',
    'сво',
    'кандидатур',
    'снял',
    'руководител',
    'федерац',
    'биатлон',
    'москв',
    'владимир',
    'малин',
    'виц',
    'президент',
    'сбр',
    'виктор',
    'майгур',
    'двукратн',
    'олимпийск',
    'чемпион',
    'дмитр',
    'васил',
    'заслужен',
    'тренер',
   

In [97]:
train_concat[2]

{'class': 'culture',
 'text': [['саш', 'барон', 'коэн', 'сыгра', 'саддам', 'хусейн'],
  ['британск',
   'комик',
   'саш',
   'барон',
   'коэн',
   'исполн',
   'рол',
   'иракск',
   'диктатор',
   'саддам',
   'хусейн',
   'сообща',
   'агентств',
   'associated',
   'press',
   'ссылк',
   'компан',
   'paramount',
   'pictures'],
  ['коэн',
   'сыгра',
   'главн',
   'рол',
   'фильм',
   'диктатор',
   'выход',
   'котор',
   'экра',
   'запланирова',
   'ма',
   '2012',
   'год',
   'примет',
   'участ',
   'написан',
   'сценар'],
  ['основ',
   'фильм',
   'полож',
   'книг',
   'забиб',
   'цар',
   'авторств',
   'котор',
   'приписыва',
   'сам',
   'саддам',
   'хусейн',
   'хот',
   'един',
   'мнен',
   'эт',
   'повод'],
  ['год',
   'правлен',
   'хусейн',
   'рома',
   'экранизирова',
   'иракск',
   'телевиден',
   'кром',
   'мотив',
   'поставл',
   'мюзикл'],
  ['режиссер', 'картин', 'станет', 'ларр', 'чарльз'],
  ['довод',
   'работа',
   'саш',
   'барон',
   'к

##  W2V и семантические ассоциации

In [98]:
from gensim.models import Word2Vec
all_sent = [sentance for item in train_data for sentance in item["text"]]
model = Word2Vec(all_sent, size=500, window=10 ,min_count=1, workers=2)

In [99]:
print(model.wv.most_similar(positive=["джеймс"]))

[('майкл', 0.9495384097099304), ('джон', 0.9455983638763428), ('питер', 0.9454224705696106), ('драм', 0.9363150596618652), ('дженнифер', 0.9345179200172424), ('капр', 0.9324065446853638), ('джонсон', 0.9305950999259949), ('комед', 0.9286341667175293), ('мартин', 0.9270502924919128), ('триллер', 0.9255245327949524)]


In [127]:
print(model.wv.most_similar(positive=["университет", "санкт", "петербург"]))

[('ленинградск', 0.8552278280258179), ('новосибирск', 0.8272430300712585), ('петергбургск', 0.8240901231765747), ('екатеринбург', 0.8002661466598511), ('выставочн', 0.7967272400856018), ('ленобласт', 0.7816773653030396), ('нерп', 0.7778173685073853), ('градостроительств', 0.7762106657028198), ('соболев', 0.7690498232841492), ('новгород', 0.7657777070999146)]


In [139]:
print(model.wv.most_similar(positive=["компьютер","экран"]))

[('гаджет', 0.9591737389564514), ('устройств', 0.9412131309509277), ('ос', 0.9362700581550598), ('модем', 0.9228483438491821), ('планшет', 0.9221930503845215), ('браузер', 0.9221010208129883), ('android', 0.919352650642395), ('смартфон', 0.9162595272064209), ('5s', 0.9113731384277344), ('ноутбук', 0.9107376337051392)]


##  TF-IDF from W2V 

In [160]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
import numpy as np

w2v = dict(zip(model.wv.index2word, model.wv.syn0))

class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        self.dim = len(next(iter(word2vec.values())))

    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])
        return self

    def transform(self, X):
        return np.array([
                np.mean([self.word2vec[w] * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """


In [161]:
train_X = []
train_Y = []
test_X = []
test_Y = []

for x in train_concat: 
    train_X.append([j for i in x['text'] for j in i])
    train_Y.append(x['class'])
    
for x in test_concat: 
    test_X.append([j for i in x['text'] for j in i])
    test_Y.append(x['class'])

##  SVM Classifier

In [118]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

svm_tfidf = Pipeline([
    ("w2v tf-idf", TfidfEmbeddingVectorizer(w2v)),
    ('scaler', StandardScaler()),
    ("SVM", SVC(kernel='linear', C=1,verbose=1))])   

svm_tfidf.fit(train_X,train_Y)

[LibSVM]

Pipeline(memory=None,
         steps=[('w2v tf-idf',
                 <__main__.TfidfEmbeddingVectorizer object at 0x1a3dfd37d0>),
                ('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('SVM',
                 SVC(C=1, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,
                     gamma='scale', kernel='linear', max_iter=-1,
                     probability=False, random_state=None, shrinking=True,
                     tol=0.001, verbose=1))],
         verbose=False)

In [119]:
svm_tfidf.score(train_X,train_Y)

0.88

In [120]:
predicts = svm_tfidf.predict(test_X)
print(f"accuracy = {(test_Y == predicts).mean()}")

accuracy = 0.8533333333333334


##  Logistic Regression

In [158]:
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA

bs_tfidf = Pipeline([
    ("w2v tf-idf", TfidfEmbeddingVectorizer(w2v)),
    ('scaler', StandardScaler()),
    ('PCA', PCA(n_components=100)),
    ("SVM", LogisticRegression(max_iter=10000))])    
bs_tfidf.fit(train_X,train_Y)

bs_tfidf.score(train_X,train_Y)

0.8548

In [159]:
predicts = bs_tfidf.predict(test_X)
print(f"accuracy = {(test_Y == predicts).mean()}")

accuracy = 0.8496666666666667
